Faradias Izza Azzahra
<br>
H071191030
<br>
Sumber data: https://www.kaggle.com/alexmiles/grocery-products-purchase-data

In [1]:
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
df_transactions = pd.read_csv("Grocery Products.csv")
df_transactions.head()

,citrus fruit,semi-finished bread,margarine,ready soups,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,whole milk,butter,yogurt,rice,abrasive cleaner,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Transformasi data 

In [3]:
#Replace semua karakter non-alphanumeric dan spasi dengan underscore

df_transactions = df_transactions.replace(r'[^A-z0-9\n]+','_', regex=True)

In [4]:
# Menggabungkan kolom pada setiap dalam bentuk CSV

x = df_transactions.to_string(header=False, index=False, index_names=False).split('\n')
vals = [','.join(e.split()) for e in x]
df_transactions['combined'] = [','.join(e.split()) for e in x]

#Hilangkan nilai NaN dari dataset 
df_transactions_tmp = df_transactions[['combined']].replace(to_replace=r',NaN.*$', value='', regex=True)

In [5]:
# Menambahkan id 

df_transactions_tmp['orderId'] = df_transactions_tmp.index

In [6]:
#Split semua transaksi kedalam baris dengan koma

df_transactions_tmp = df_transactions_tmp.assign(combined=df_transactions_tmp.combined.str.split(',')).explode('combined').reset_index(drop=True)

#Replace temporary underscore dengan spasi
df_transactions_tmp = df_transactions_tmp.replace(r'_',' ', regex=True)

In [7]:
#Menentukan kuantitas transaksi dengan memberikan nilai '1' untuk setiap kolom

df_transactions_tmp['itemQuantity'] = 1

In [8]:
#Menampilkan dataframe

df = df_transactions_tmp[['orderId','combined','itemQuantity']]
df.columns = ['orderId','itemDescription','itemQuantity']
df

,orderId,itemDescription,itemQuantity
0,0,tropical fruit,1
1,0,yogurt,1
2,0,coffee,1
3,1,whole milk,1
4,2,pip fruit,1
...,...,...,...
43358,9833,chicken,1
43359,9833,tropical fruit,1
43360,9833,other vegetables,1
43361,9833,vinegar,1


In [9]:
#Menggabungkan produk dengan 'orderId' yang sama (grouping)
df_grouped = df.groupby(['orderId','itemDescription']).sum()['itemQuantity']
df_grouped

orderId  itemDescription 
0        coffee              1
         tropical fruit      1
         yogurt              1
1        whole milk          1
2        cream cheese        1
                            ..
9833     chicken             1
         other vegetables    1
         shopping bags       1
         tropical fruit      1
         vinegar             1
Name: itemQuantity, Length: 43363, dtype: int64

In [10]:
#Menampilkan data untuk setiap transaksi

df_basket = df_grouped.unstack().reset_index().fillna(0).set_index('orderId')
df_basket

itemDescription,Instant food products,UHT milk,abrasive cleaner,artif sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
orderId,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Encoding Nilai Data 

Data di encode ke dalam nilai 1 dan 0, dimana 1 bernilai True dan 0 bernilai False menunjukkan produk yang dibeli dan tidak dibeli

In [11]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = df_basket.applymap(encode_units)
basket_sets.head()

itemDescription,Instant food products,UHT milk,abrasive cleaner,artif sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
orderId,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


Algoritma Apriori

In [12]:
#Menampilkan item dengan minimum support 0.03

frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True).sort_values(by=['support'], ascending=False)
frequent_itemsets.head()

,support,itemsets
42,0.255542,(whole milk)
26,0.193512,(other vegetables)
30,0.183954,(rolls buns)
35,0.174395,(soda)
43,0.139516,(yogurt)


Data diatas menunjukkan bahwa susu, sayuran, dan roti merupakan item paling populer di market.

In [13]:
#Membuat hasil rules dari algoritma Apriori

rules = association_rules(frequent_itemsets, metric="lift")
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(other vegetables),(whole milk),0.193512,0.255542,0.074842,0.386758,1.513480,0.025392,1.213971
1,(whole milk),(other vegetables),0.255542,0.193512,0.074842,0.292877,1.513480,0.025392,1.140520
2,(rolls buns),(whole milk),0.183954,0.255542,0.056640,0.307905,1.204909,0.009632,1.075659
3,(whole milk),(rolls buns),0.255542,0.183954,0.056640,0.221647,1.204909,0.009632,1.048428
4,(yogurt),(whole milk),0.139516,0.255542,0.056030,0.401603,1.571575,0.020378,1.244088
5,(whole milk),(yogurt),0.255542,0.139516,0.056030,0.219260,1.571575,0.020378,1.102139
6,(root vegetables),(whole milk),0.109010,0.255542,0.048912,0.448694,1.755852,0.021055,1.350354
7,(whole milk),(root vegetables),0.255542,0.109010,0.048912,0.191405,1.755852,0.021055,1.101899
8,(root vegetables),(other vegetables),0.109010,0.193512,0.047387,0.434701,2.246376,0.026292,1.426658
9,(other vegetables),(root vegetables),0.193512,0.109010,0.047387,0.244877,2.246376,0.026292,1.179927


In [14]:
#Menampilkan rules dari algoritma Apriori dengan filter lift dengan minimum treshold = 1, dan confidence lebih dari 25%

rules[ (rules['lift'] > 1) & (rules['confidence'] >= 0.25) ].sort_values('lift', ascending=True).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(rolls buns),(whole milk),0.183954,0.255542,0.056640,0.307905,1.204909,0.009632,1.075659
1,(bottled water),(whole milk),0.110535,0.255542,0.034371,0.310948,1.216816,0.006124,1.080408
2,(citrus fruit),(whole milk),0.082672,0.255542,0.030506,0.369004,1.444004,0.009380,1.179814
3,(pastry),(whole milk),0.088977,0.255542,0.033252,0.373714,1.462438,0.010515,1.188688
4,(other vegetables),(whole milk),0.193512,0.255542,0.074842,0.386758,1.513480,0.025392,1.213971
5,(whole milk),(other vegetables),0.255542,0.193512,0.074842,0.292877,1.513480,0.025392,1.140520
6,(pip fruit),(whole milk),0.075656,0.255542,0.030100,0.397849,1.556885,0.010766,1.236332
7,(yogurt),(whole milk),0.139516,0.255542,0.056030,0.401603,1.571575,0.020378,1.244088
8,(tropical fruit),(whole milk),0.104942,0.255542,0.042302,0.403101,1.577435,0.015485,1.247209
9,(yogurt),(other vegetables),0.139516,0.193512,0.043421,0.311224,1.608293,0.016423,1.170901
